# US Name

Data preparation combine table information and journals


# Description

None

## Merge

**Main table** 

papers_meta_analysis_new

Merged with:

- papers_meta_analysis
- journals_scimago

# Target

- The file is saved in S3:
- bucket: datalake-datascience
- path: DATA/FINANCE/ESG/ESG_CFP
- Glue data catalog should be updated
- database: esg
- Table prefix: meta_analysis_
- table name: meta_analysis_esg_cfp
- Analytics
- HTML: ANALYTICS/HTML_OUTPUT/meta_analysis_esg_cfp
- Notebook: ANALYTICS/OUTPUT/meta_analysis_esg_cfp

# Metadata

- Key: 234_esg_metadata
- Epic: Dataset transformation
- US: Prepare meta-analysis table
- Task tag: #journal-information, #papers-information
- Analytics reports: https://htmlpreview.github.io/?https://github.com/thomaspernet/esg_metadata/blob/master/00_data_catalog/HTML_ANALYSIS/META_ANALYSIS_ESG_CFP.html

# Input

## Table/file

**Name**

- papers_meta_analysis_new
- papers_meta_analysis
- journals_scimago

**Github**

- https://github.com/thomaspernet/esg_metadata/blob/master/01_data_preprocessing/01_transform_tables/00_meta_analysis.md

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_s3 import service_s3
from awsPy.aws_glue import service_glue
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
import os, shutil, json, re

path = os.getcwd()
parent_path = str(Path(path).parent.parent)


name_credential = 'financial_dep_SO2_accessKeys.csv'
region = 'eu-west-2'
bucket = 'datalake-london'
path_cred = "{0}/creds/{1}".format(parent_path, name_credential)

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = True) 
glue = service_glue.connect_glue(client = client) 

In [3]:
pandas_setting = True
if pandas_setting:
    cm = sns.light_palette("green", as_cmap=True)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Prepare query 

Write query and save the CSV back in the S3 bucket `datalake-datascience` 

# Steps

## Merge journal and papers table

In [4]:
DatabaseName = 'esg'
s3_output_example = 'SQL_OUTPUT_ATHENA'

In [5]:
query = """
WITH merge AS (
  SELECT 
    id, 
    image,
    row_id_excel,
    table_refer,
    row_id_google_spreadsheet,
    incremental_id,
    paper_name, 
    publication_year, 
    publication_type, 
    regexp_replace(
      regexp_replace(
        lower(publication_name), 
        '\&', 
        'and'
      ), 
      '\-', 
      ' '
    ) as publication_name, 
    cnrs_ranking, 
    peer_reviewed, 
    study_focused_on_social_environmental_behaviour, 
    type_of_data, 
    study_focusing_on_developing_or_developed_countries,
    regions,
    first_date_of_observations,
    last_date_of_observations,
    adjusted_model_name,
    adjusted_model,
    dependent, 
    adjusted_dependent,
    independent,
    adjusted_independent, 
    social,
    environmental,
    governance,
    sign_of_effect,
    target,
    lag, 
    interaction_term, 
    quadratic_term, 
    n, 
    r2, 
    beta, 
    to_remove, 
    critical_value, 
    true_standard_error, 
    final_standard_error,
    true_t_value, 
    true_stars,  
    significant,
    to_check_final
  FROM 
    esg.papers_meta_analysis_new 
    LEFT JOIN (
      SELECT 
        DISTINCT(title),
        nr, 
        publication_year, 
        publication_type, 
        publication_name, 
        cnrs_ranking, 
        peer_reviewed, 
        study_focused_on_social_environmental_behaviour, 
        type_of_data, 
        study_focusing_on_developing_or_developed_countries
      FROM 
        esg.papers_meta_analysis
    ) as old on papers_meta_analysis_new.id = old.nr
    -- WHERE to_remove = 'TO_KEEP'
LEFT JOIN (
SELECT 
        nr,
        CAST(MIN(first_date_of_observations) as int) as first_date_of_observations,
        CAST(MAX(last_date_of_observations)as int) as last_date_of_observations,
        min(row_id_excel) as row_id_excel
      FROM 
        esg.papers_meta_analysis
        GROUP BY nr
) as date_pub on papers_meta_analysis_new.id = date_pub.nr
LEFT JOIN (
SELECT 
  nr, 
  MIN(regions) as regions 
FROM 
  (
    SELECT 
      nr, 
      CASE WHEN regions_of_selected_firms in (
        'Cameroon', 'Egypt', 'Libya', 'Morocco', 
        'Nigeria'
      ) THEN 'AFRICA' WHEN regions_of_selected_firms in ('GCC countries') THEN 'ARAB WORLD' WHEN regions_of_selected_firms in (
        'India', 'Indonesia', 'Taiwan', 'Vietnam', 
        'Australia', 'China', 'Iran', 'Malaysia', 
        'Pakistan', 'South Korea', 'Bangladesh'
      ) THEN 'ASIA AND PACIFIC' WHEN regions_of_selected_firms in (
        'Spain', '20 European countries', 
        'United Kingdom', 'France', 'Germany, Italy, the Netherlands and United Kingdom', 
        'Turkey', 'UK'
      ) THEN 'EUROPE' WHEN regions_of_selected_firms in ('Latin America', 'Brazil') THEN 'LATIN AMERICA' WHEN regions_of_selected_firms in ('USA', 'US', 'U.S.', 'Canada') THEN 'NORTH AMERICA' ELSE 'WORLDWIDE' END AS regions 
    FROM 
      papers_meta_analysis
  ) 
GROUP BY 
  nr
) as reg on papers_meta_analysis_new.id = reg.nr
) 
SELECT 
    to_remove, 
    id, 
    image,
    row_id_excel,
    row_id_google_spreadsheet,
    table_refer,
    incremental_id,
    paper_name,
    publication_name,
    rank,
    sjr, 
    sjr_best_quartile, 
    h_index, 
    total_docs_2020, 
    total_docs_3years, 
    total_refs, 
    total_cites_3years, 
    citable_docs_3years, 
    cites_doc_2years, 
    country ,
    publication_year, 
    publication_type, 
    cnrs_ranking, 
    peer_reviewed, 
    study_focused_on_social_environmental_behaviour, 
    type_of_data, 
    study_focusing_on_developing_or_developed_countries, 
    regions,
    first_date_of_observations,
    last_date_of_observations,
    CASE WHEN first_date_of_observations >= 1997 THEN TRUE ELSE FALSE END AS kyoto,
    CASE WHEN first_date_of_observations >= 2009 THEN TRUE ELSE FALSE END AS financial_crisis,
    last_date_of_observations - first_date_of_observations as windows,
    (last_date_of_observations - first_date_of_observations) / 2 avg_windows,
    adjusted_model_name,
    adjusted_model, 
    dependent, 
    adjusted_dependent, 
    independent, 
    adjusted_independent, 
    social,
    environmental,
    governance,
    lag, 
    interaction_term, 
    quadratic_term, 
    n, 
    r2, 
    beta, 
    sign_of_effect,
    target,
    significant,
    final_standard_error,
    to_check_final 
FROM 
  merge 
  LEFT JOIN (
    SELECT 
      rank, 
      regexp_replace(
        regexp_replace(
          lower(title), 
          '\&', 
          'and'
        ), 
        '\-', 
        ' '
      ) as title, 
      sjr, 
      sjr_best_quartile, 
      h_index, 
      total_docs_2020, 
      total_docs_3years, 
      total_refs, 
      total_cites_3years, 
      citable_docs_3years, 
      cites_doc_2years, 
      country 
    FROM 
      "scimago"."journals_scimago"
    WHERE sourceid not in (16400154787)
  ) as journal on merge.publication_name = journal.title
"""
output = (
    s3.run_query(
    query=query,
    database=DatabaseName,
    s3_output=s3_output_example,
    filename='example_1',
        dtype = {'publication_year':'string'}
)
    .sort_values(by = ['id', 'first_date_of_observations'])
    .drop_duplicates()
    .assign(weight = lambda x: x.groupby(['id'])['id'].transform('size'))
)
output.head()

,to_remove,id,image,row_id_excel,row_id_google_spreadsheet,table_refer,incremental_id,paper_name,publication_name,rank,sjr,sjr_best_quartile,h_index,total_docs_2020,total_docs_3years,total_refs,total_cites_3years,citable_docs_3years,cites_doc_2years,country,publication_year,publication_type,cnrs_ranking,peer_reviewed,study_focused_on_social_environmental_behaviour,type_of_data,study_focusing_on_developing_or_developed_countries,regions,first_date_of_observations,last_date_of_observations,kyoto,financial_crisis,windows,avg_windows,adjusted_model_name,adjusted_model,dependent,adjusted_dependent,independent,adjusted_independent,social,environmental,governance,lag,interaction_term,quadratic_term,n,r2,beta,sign_of_effect,target,significant,final_standard_error,to_check_final,weight
0,TO_KEEP,1,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=479654123&range=B2,https://docs.google.com/spreadsheets/d/11A3l50yfiGxxRuyV-f3WV9Z-4DcsQLYW6XBl4a7U4bQ/edit#gid=18346659&range=B10,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650&range=B2,table 4,1,Corporate social responsibility financial instability and corporate,finance research letters,2728.0,1.339,Q1,39.0,442.0,567.0,11373.0,2909.0,566.0,5.47,Netherlands,2020.0,Journal Article,3.0,Yes,Environmental Social and Governance,Panel data,Developed,EUROPE,2002.0,2017.0,True,False,15.0,7.0,GMM,OTHER,ROA,ROA,CSR,CSR,True,False,False,no,no,no,469.0,0.108,0.0122,INSIGNIFICANT,NOT_SIGNIFICANT,NO,0.0135,NO,12
1,TO_KEEP,1,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=479654123&range=B2,https://docs.google.com/spreadsheets/d/11A3l50yfiGxxRuyV-f3WV9Z-4DcsQLYW6XBl4a7U4bQ/edit#gid=18346659&range=B10,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650&range=B3,table 4,2,Corporate social responsibility financial instability and corporate,finance research letters,2728.0,1.339,Q1,39.0,442.0,567.0,11373.0,2909.0,566.0,5.47,Netherlands,2020.0,Journal Article,3.0,Yes,Environmental Social and Governance,Panel data,Developed,EUROPE,2002.0,2017.0,True,False,15.0,7.0,GMM,OTHER,ROA,ROA,CSR,CSR,True,False,False,no,no,no,469.0,0.198,1.2115,POSITIVE,SIGNIFICANT,YES,0.1581,NO,12
2,TO_KEEP,1,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=479654123&range=B2,https://docs.google.com/spreadsheets/d/11A3l50yfiGxxRuyV-f3WV9Z-4DcsQLYW6XBl4a7U4bQ/edit#gid=18346659&range=B10,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650&range=B4,table 4,3,Corporate social responsibility financial instability and corporate,finance research letters,2728.0,1.339,Q1,39.0,442.0,567.0,11373.0,2909.0,566.0,5.47,Netherlands,2020.0,Journal Article,3.0,Yes,Environmental Social and Governance,Panel data,Developed,EUROPE,2002.0,2017.0,True,False,15.0,7.0,GMM,OTHER,ROE,ROE,CSR,CSR,True,False,False,no,no,no,469.0,0.857,0.3959,POSITIVE,SIGNIFICANT,YES,0.0800,NO,12
3,TO_KEEP,1,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=479654123&range=B2,https://docs.google.com/spreadsheets/d/11A3l50yfiGxxRuyV-f3WV9Z-4DcsQLYW6XBl4a7U4bQ/edit#gid=18346659&range=B10,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650&range=B5,table 4,4,Corporate social responsibility financial instability and corporate,finance research letters,2728.0,1.339,Q1,39.0,442.0,567.0,11373.0,2909.0,566.0,5.47,Netherlands,2020.0,Journal Article,3.0,Yes,Environmental Social and Governance,Panel data,Developed,EUROPE,2002.0,2017.0,True,False,15.0,7.0,GMM,OTHER,ROE,ROE,CSR,CSR,True,False,False,no,no,no,469.0,0.375,6.8766,POSITIVE,SIGNIFICANT,YES,0.5481,NO,12
4,TO_KEEP,1,https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=479654123&range=B2,https://docs.google.com/spreadsheets/d/1

In [6]:
output.shape

(701, 55)

In [7]:
output.describe()

,id,incremental_id,rank,sjr,h_index,total_docs_2020,total_docs_3years,total_refs,total_cites_3years,citable_docs_3years,cites_doc_2years,cnrs_ranking,first_date_of_observations,last_date_of_observations,windows,avg_windows,n,r2,beta,final_standard_error,weight
count,701.000000,701.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,597.000000,695.000000,689.000000,689.000000,689.000000,689.000000,692.000000,590.000000,698.000000,561.000000,701.000000
mean,49.251070,351.000000,3128.402010,2.651288,132.236181,793.857621,1427.798995,50777.407035,9698.144054,1415.222781,6.619229,1.109353,2002.126270,2009.747460,7.621190,3.557329,1507.518786,1.330399,2.518595,2.042177,19.610556
std,30.071862,202.505555,3570.007939,2.792219,79.331114,2035.111910,3104.124126,125558.776862,21323.034999,3085.183170,3.019530,1.227500,6.877036,6.084442,4.628986,2.285567,3134.446149,7.279848,57.714197,26.749121,19.628430
min,1.000000,1.000000,59.000000,0.314000,16.000000,23.000000,52.000000,1741.000000,69.000000,50.000000,1.390000,0.000000,1972.000000,1986.000000,0.000000,0.000000,20.000000,-0.014500,-355.000000,-9.180000,1.000000
25%,28.000000,176.000000,652.000000,1.115000,73.000000,152.000000,282.000000,9601.000000,2331.000000,280.000000,3.750000,0.000000,1999.000000,2006.000000,4.000000,2.000000,183.250000,0.130000,-0.006000,0.004000,6.000000
50%,45.000000,351.000000,1743.000000,1.734000,103.000000,230.000000,509.000000,18613.000000,3418.000000,490.000000,7.380000,1.000000,2004.000000,2011.000000,6.000000,3.000000,564.000000,0.259000,0.008000,0.049096,12.000000
75%,79.000000,526.000000,3714.000000,2.983000,199.000000,439.000000,891.000000,20680.000000,8098.000000,881.000000,8.980000,2.000000,2007.000000,2014.000000,10.000000,5.000000,937.000000,0.444800,0.152750,0.523557,24.000000
max,110.000000,701.000000,14377.000000,11.193000,318.000000,10654.000000,14468.000000,664559.000000,103295.000000,14339.000000,16.300000,4.000000,2018.000000,2018.000000,29.000000,14.000000,26482.000000,57.310000,1451.910000,628.720000,70.000000


In [8]:
output['weight'].describe()

count    701.000000
mean      19.610556
std       19.628430
min        1.000000
25%        6.000000
50%       12.000000
75%       24.000000
max       70.000000
Name: weight, dtype: float64

In [9]:
#output[output.duplicated(subset = ['id', 'beta',
#                                   'true_standard_error', 'critical_value', 'lag', 'independent',
#                                  'true_t_value', 'true_stars', 'adjusted_model'
#                                  ])].head()

Missing journals

In [10]:
output.loc[lambda x: x['rank'].isin([np.nan])]['publication_name'].unique()

array(['the north american journal of economics and finance',
       'economic research ekonomska istraživanja',
       'journal of business studies quarterly',
       'asian journal of sustainability and social responsibility',
       'the british accounting review',
       'journal of business and management',
       'corporate ownership and control',
       'la revue des sciences de gestion', 'revue congolaise de gestion',
       'jb learning llc',
       'international journal of management and marketing research',
       'issues in social and environmental accounting',
       'globsyn management journal', 'ie business school working paper',
       'management and labour studies', 'accounting forum elsevier',
       'international journal of value based management'], dtype=object)

Currently, the missing values come from the rows to check in [METADATA_TABLES_COLLECTION](https://docs.google.com/spreadsheets/d/1d66_CVtWni7wmKlIMcpaoanvT2ghmjbXARiHgnLWvUw/edit#gid=899172650)

In [11]:
output.isna().sum().loc[lambda x: x> 0].sort_values()

dependent                                                2
independent                                              2
adjusted_dependent                                       2
beta                                                     3
table_refer                                              4
quadratic_term                                           5
interaction_term                                         5
lag                                                      5
cnrs_ranking                                             6
publication_type                                         8
type_of_data                                             8
n                                                        9
study_focusing_on_developing_or_developed_countries     10
last_date_of_observations                               12
first_date_of_observations                              12
avg_windows                                             12
windows                                                 

## Explain missings:

### Date

- 'Does Corporate Social Responsibility Pay Off in Times of Crisis? An Alternate Perspective on the Relationship between Financial and Corporate Social Performance':
    - No date
- 'How does corporate social responsibility contribute to firm financial performance? The mediating role of competitive advantage reputation and customer satisfaction',
    - No date
- 'L’impact de la responsabilité sociale (RSE) sur la performance financière des entreprises (PFE) au Cameroun'
    - Poor date formating: 2007 Semester I, 2007 Semester I

In [12]:
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

In [13]:
(
    output
    .loc[lambda x: x['first_date_of_observations'].isin([np.nan])]
    .reindex(columns = ['paper_name', 'url_excel'])
    .drop_duplicates()
    .style
    .format(make_clickable, subset = ['url_excel'])
)

,paper_name,url_excel
385,How does corporate social responsibility contribute to firm financial performance? The mediating role of competitive advantage reputation and customer satisfaction,nan
397,L’impact de la responsabilité sociale (RSE) sur la performance financière des entreprises (PFE) au Cameroun,nan
451,Does Corporate Social Responsibility Pay Off in Times of Crisis? An Alternate Perspective on the Relationship between Financial and Corporate Social Performance,nan


### location

- Corporate social responsibility and financial performance in Islamic banks
    - Missing
- Corporate social responsibility and financial performance: the ‘‘virtuous circle’’ revisited
    - Missing

In [14]:
(
    output
    .loc[lambda x: x['study_focusing_on_developing_or_developed_countries'].isin([np.nan])]
    .reindex(columns = ['paper_name', 'url_excel'])
    .drop_duplicates()
    .style
    .format(make_clickable, subset = ['url_excel'])
)

,paper_name,url_excel
391,Corporate social responsibility and financial performance in Islamic banks,nan
593,Corporate social responsibility and financial performance: the ‘‘virtuous circle’’ revisited,nan


### peer_reviewed

- L’impact de la responsabilité sociale (RSE) sur la performance financière des entreprises (PFE) au Cameroun
    - Missing
- Looking for evidence of the relationship between CSR and CFP in an Emerging Market
    - Missing
- The Relationship of CSR and Financial Performance: New Evidence From Indonesian Companies
    - Missing
- Exploring the moderating effect of financial performance on the relationship between corporate environmental responsibility and institutional investors: some Egyptian evidence
    - Missing
- STRATEGIC USE OF CSR AS A SIGNAL FOR GOOD MANAGEMENT
    - Missing

In [15]:
(
    output
    .loc[lambda x: x['peer_reviewed'].isin([np.nan])]
    .reindex(columns = ['paper_name', 'url_excel'])
    .drop_duplicates()
    .style
    .format(make_clickable, subset = ['url_excel'])
)

,paper_name,url_excel
397,L’impact de la responsabilité sociale (RSE) sur la performance financière des entreprises (PFE) au Cameroun,nan
440,Looking for evidence of the relationship between CSR and CFP in an Emerging Market,nan
562,The Relationship of CSR and Financial Performance: New Evidence From Indonesian Companies,nan
625,Exploring the moderating effect of financial performance on the relationship between corporate environmental responsibility and institutional investors: some Egyptian evidence,nan
636,STRATEGIC USE OF CSR AS A SIGNAL FOR GOOD MANAGEMENT,nan


## adjusted_independent

- Need to be adjusted

In [16]:
(
    output
    .loc[lambda x: x['adjusted_independent'].isin([np.nan])]
    .reindex(columns = ['paper_name', 'url_google_spreadsheet'])
    .drop_duplicates()
    .head(5)
    .style
    .format(make_clickable, subset = ['url_google_spreadsheet'])
)

,paper_name,url_google_spreadsheet
25,The heterogeneous effects of CSR dimensions on financial performance a new approach for CSR measurement,nan
121,Environmental Social and Governance (ESG) Scores and Financial Performance of Multilatinas: Moderating Effects of Geographic International Diversification and Financial Slack,nan
398,L’impact de la responsabilité sociale (RSE) sur la performance financière des entreprises (PFE) au Cameroun,nan
412,The CSR-Firm Performance Missing Link: Complementarity Between Environmental Social and Business Behavior Criteria?,nan
448,CORPORATE SOCIAL RESPONSIBILITY AND SHAREHOLDER REACTION the environmental awareness of investors,nan


### Save data to Google Spreadsheet for sharing

- Link: [METADATA_MODEL](https://docs.google.com/spreadsheets/d/13gpRy93l7POWGe-rKjytt7KWOcD1oSLACngTEpuqCTg/edit#gid=0)

In [17]:
#!pip install --upgrade git+git://github.com/thomaspernet/GoogleDrive-python

In [18]:
from GoogleDrivePy.google_drive import connect_drive
from GoogleDrivePy.google_authorization import authorization_service

In [19]:
try:
    os.mkdir("creds")
except:
    pass

In [20]:
s3.download_file(key = "CREDS/Financial_dependency_pollution/creds/token.pickle", path_local = "creds")

In [21]:
auth = authorization_service.get_authorization(
    #path_credential_gcp=os.path.join(parent_path, "creds", "service.json"),
    path_credential_drive=os.path.join(path, "creds"),
    verbose=False,
    scope=['https://www.googleapis.com/auth/spreadsheets.readonly',
           "https://www.googleapis.com/auth/drive"]
)
gd_auth = auth.authorization_drive(path_secret=os.path.join(
    path, "creds", "credentials.json"))
drive = connect_drive.drive_operations(gd_auth)

In [22]:
import shutil
shutil.rmtree(os.path.join(path,"creds"))

In [23]:
FILENAME_SPREADSHEET = "METADATA_MODEL"
spreadsheet_id = drive.find_file_id(FILENAME_SPREADSHEET, to_print=False)

In [24]:
drive.add_data_to_spreadsheet(
    data =output.fillna(""),
    sheetID =spreadsheet_id,
    sheetName = "MODEL_DATA",
    detectRange = True,
    rangeData = None)

Range MODEL_DATA!A1:BD705 has been updated.


# Table `meta_analysis_esg_cfp`

Since the table to create has missing value, please use the following at the top of the query

```
CREATE TABLE database.table_name WITH (format = 'PARQUET') AS
```

Choose a location in S3 to save the CSV. It is recommended to save in it the `datalake-datascience` bucket. Locate an appropriate folder in the bucket, and make sure all output have the same format

In [25]:
s3_output = 'DATA/FINANCE/ESG/ESG_CFP'
table_name = 'meta_analysis_esg_cfp'

First, we need to delete the table (if exist)

In [26]:
try:
    response = glue.delete_table(
        database=DatabaseName,
        table=table_name
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': '8d1c91f8-04dd-4b6c-9429-5def5f36bce0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 06 Aug 2021 09:48:44 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'connection': 'keep-alive', 'x-amzn-requestid': '8d1c91f8-04dd-4b6c-9429-5def5f36bce0'}, 'RetryAttempts': 0}}


Clean up the folder with the previous csv file. Be careful, it will erase all files inside the folder

In [27]:
s3.remove_all_bucket(path_remove = s3_output)

True

In [28]:
%%time
query = """
CREATE TABLE {0}.{1} WITH (format = 'PARQUET') AS
WITH merge AS (
  SELECT 
    id, 
    image,
    row_id_excel,
    table_refer,
    row_id_google_spreadsheet,
    incremental_id,
    paper_name, 
    publication_year, 
    publication_type, 
    regexp_replace(
      regexp_replace(
        lower(publication_name), 
        '\&', 
        'and'
      ), 
      '\-', 
      ' '
    ) as publication_name, 
    cnrs_ranking, 
    peer_reviewed, 
    study_focused_on_social_environmental_behaviour, 
    type_of_data, 
    study_focusing_on_developing_or_developed_countries,
    regions,
    first_date_of_observations,
    last_date_of_observations,
    adjusted_model_name,
    adjusted_model,
    dependent, 
    adjusted_dependent,
    independent,
    adjusted_independent, 
    social,
    environmental,
    governance,
    sign_of_effect,
    target,
    lag, 
    interaction_term, 
    quadratic_term, 
    n, 
    r2, 
    beta, 
    to_remove, 
    critical_value, 
    true_standard_error, 
    final_standard_error,
    true_t_value, 
    true_stars,  
    significant,
    to_check_final
  FROM 
    esg.papers_meta_analysis_new 
    LEFT JOIN (
      SELECT 
        DISTINCT(title),
        nr, 
        publication_year, 
        publication_type, 
        publication_name, 
        cnrs_ranking, 
        peer_reviewed, 
        study_focused_on_social_environmental_behaviour, 
        type_of_data, 
        study_focusing_on_developing_or_developed_countries
      FROM 
        esg.papers_meta_analysis
    ) as old on papers_meta_analysis_new.id = old.nr
    -- WHERE to_remove = 'TO_KEEP'
LEFT JOIN (
SELECT 
        nr,
        CAST(MIN(first_date_of_observations) as int) as first_date_of_observations,
        CAST(MAX(last_date_of_observations)as int) as last_date_of_observations,
        min(row_id_excel) as row_id_excel
      FROM 
        esg.papers_meta_analysis
        GROUP BY nr
) as date_pub on papers_meta_analysis_new.id = date_pub.nr
LEFT JOIN (
SELECT 
  nr, 
  MIN(regions) as regions 
FROM 
  (
    SELECT 
      nr, 
      CASE WHEN regions_of_selected_firms in (
        'Cameroon', 'Egypt', 'Libya', 'Morocco', 
        'Nigeria'
      ) THEN 'AFRICA' WHEN regions_of_selected_firms in ('GCC countries') THEN 'ARAB WORLD' WHEN regions_of_selected_firms in (
        'India', 'Indonesia', 'Taiwan', 'Vietnam', 
        'Australia', 'China', 'Iran', 'Malaysia', 
        'Pakistan', 'South Korea', 'Bangladesh'
      ) THEN 'ASIA AND PACIFIC' WHEN regions_of_selected_firms in (
        'Spain', '20 European countries', 
        'United Kingdom', 'France', 'Germany, Italy, the Netherlands and United Kingdom', 
        'Turkey', 'UK'
      ) THEN 'EUROPE' WHEN regions_of_selected_firms in ('Latin America', 'Brazil') THEN 'LATIN AMERICA' WHEN regions_of_selected_firms in ('USA', 'US', 'U.S.', 'Canada') THEN 'NORTH AMERICA' ELSE 'WORLDWIDE' END AS regions 
    FROM 
      papers_meta_analysis
  ) 
GROUP BY 
  nr
) as reg on papers_meta_analysis_new.id = reg.nr
) 
SELECT 
    to_remove, 
    id, 
    image,
    row_id_excel,
    row_id_google_spreadsheet,
    table_refer,
    incremental_id,
    paper_name,
    publication_name,
    rank,
    sjr, 
    sjr_best_quartile, 
    h_index, 
    total_docs_2020, 
    total_docs_3years, 
    total_refs, 
    total_cites_3years, 
    citable_docs_3years, 
    cites_doc_2years, 
    country ,
    publication_year, 
    publication_type, 
    cnrs_ranking, 
    peer_reviewed, 
    study_focused_on_social_environmental_behaviour, 
    type_of_data, 
    study_focusing_on_developing_or_developed_countries, 
    regions,
    first_date_of_observations,
    last_date_of_observations,
    CASE WHEN first_date_of_observations >= 1997 THEN TRUE ELSE FALSE END AS kyoto,
    CASE WHEN first_date_of_observations >= 2000 THEN TRUE ELSE FALSE END AS financial_crisis,
    last_date_of_observations - first_date_of_observations as windows,
    (last_date_of_observations - first_date_of_observations) / 2 avg_windows,
    adjusted_model_name,
    adjusted_model, 
    dependent, 
    adjusted_dependent, 
    independent, 
    adjusted_independent, 
    social,
    environmental,
    governance,
    lag, 
    interaction_term, 
    quadratic_term, 
    n, 
    r2, 
    beta, 
    sign_of_effect,
    target,
    significant,
    final_standard_error,
    to_check_final 
FROM 
  merge 
  LEFT JOIN (
    SELECT 
      rank, 
      regexp_replace(
        regexp_replace(
          lower(title), 
          '\&', 
          'and'
        ), 
        '\-', 
        ' '
      ) as title, 
      sjr, 
      sjr_best_quartile, 
      h_index, 
      total_docs_2020, 
      total_docs_3years, 
      total_refs, 
      total_cites_3years, 
      citable_docs_3years, 
      cites_doc_2years, 
      country 
    FROM 
      "scimago"."journals_scimago"
    WHERE sourceid not in (16400154787)
  ) as journal on merge.publication_name = journal.title
""".format(DatabaseName, table_name)
output = s3.run_query(
                    query=query,
                    database=DatabaseName,
                    s3_output=s3_output,
                )
output

CPU times: user 145 ms, sys: 3.37 ms, total: 148 ms
Wall time: 3.02 s


{'Results': {'State': 'SUCCEEDED',
  'SubmissionDateTime': datetime.datetime(2021, 8, 6, 9, 48, 44, 682000, tzinfo=tzlocal()),
  'CompletionDateTime': datetime.datetime(2021, 8, 6, 9, 48, 47, 280000, tzinfo=tzlocal())},
 'QueryID': '6de46b3e-29c2-40f8-9f88-3902d85d47fe'}

In [29]:
query_count = """
SELECT COUNT(*) AS CNT
FROM {}.{} 
""".format(DatabaseName, table_name)
output = s3.run_query(
                    query=query_count,
                    database=DatabaseName,
                    s3_output=s3_output_example,
    filename = 'count_{}'.format(table_name)
                )
output

,CNT
0,701


# Update Glue catalogue and Github

This step is mandatory to validate the query in the ETL.

## Create or update the data catalog

The query is saved in the S3 (bucket `datalake-london`), but the comments are not available. Use the functions below to update the catalogue and Github


Update the dictionary

- DatabaseName:
- TableName:
- ~TablePrefix:~
- input: 
- filename: Name of the notebook or Python script: to indicate
- Task ID: from Coda
- index_final_table: a list to indicate if the current table is used to prepare the final table(s). If more than one, pass the index. Start at 0
- if_final: A boolean. Indicates if the current table is the final table -> the one the model will be used to be trained
- schema: glue schema with comment
- description: details query objective

**Update schema**

If `automatic = False` in `automatic_update`, then the function returns only the variables to update the comments. Manually add the comment, **then**, pass the new schema (only the missing comment) to the argument `new_schema`. 

To update the schema, please use the following structure

```
schema = [
    {
        "Name": "VAR1",
        "Type": "",
        "Comment": ""
    },
    {
        "Name": "VAR2",
        "Type": "",
        "Comment": ""
    }
]
```

In [30]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append(os.path.join(parent_path, 'utils'))
import make_toc
import create_schema
import create_report
import update_glue_github

The function below manages everything automatically. If the final table comes from more than one query, then pass a list of table in `list_tables` instead of `automatic`

In [31]:
list_input,  schema = update_glue_github.automatic_update(
    list_tables = 'automatic',
    automatic= True,
    new_schema = None, ### override schema
    client = client,
    TableName = table_name,
    query = query)

In [32]:
description = """
Create table with journal information, papers and coefficients for the meta analysis
"""
name_json = 'parameters_ETL_esg_metadata.json'
partition_keys = ["id", 'incremental_id']
notebookname = "00_meta_analysis.ipynb"
dic_information = {
    "client":client,
    'bucket':bucket,
    's3_output':s3_output,
    'DatabaseName':DatabaseName,
    'TableName':table_name,
    'name_json':name_json,
    'partition_keys':partition_keys,
    'notebookname':notebookname,
    'index_final_table':[0],
    'if_final': 'True',
    'schema':schema,
    'description':description,
    'query':query,
    "list_input":list_input,
    'list_input_automatic':True
}

In [33]:
update_glue_github.update_glue_github(client = client,dic_information = dic_information)

Currently, the ETL has 1 tables


## Check Duplicates

One of the most important step when creating a table is to check if the table contains duplicates. The cell below checks if the table generated before is empty of duplicates. The code uses the JSON file to create the query parsed in Athena. 

You are required to define the group(s) that Athena will use to compute the duplicate. For instance, your table can be grouped by COL1 and COL2 (need to be string or varchar), then pass the list ['COL1', 'COL2'] 

In [34]:
update_glue_github.find_duplicates(
    client = client,
    bucket = bucket,
    name_json = name_json,
    partition_keys = partition_keys,
    TableName= table_name
)

,CNT,CNT_DUPLICATE
0,1,701


## Count missing values

In [35]:
update_glue_github.count_missing(client = client, name_json = name_json, bucket = bucket,TableName = table_name)

,total_missing,total_missing_pct
nb_obs,701,100.00%
final_standard_error,140,19.97%
r2,111,15.83%
sjr_best_quartile,104,14.84%
country,104,14.84%
cites_doc_2years,104,14.84%
citable_docs_3years,104,14.84%
total_refs,104,14.84%
total_docs_3years,104,14.84%
total_docs_2020,104,14.84%


# Update Github Data catalog

The data catalog is available in Glue. Although, we might want to get a quick access to the tables in Github. In this part, we are generating a `README.md` in the folder `00_data_catalogue`. All tables used in the project will be added to the catalog. We use the ETL parameter file and the schema in Glue to create the README. 

Bear in mind the code will erase the previous README. 

In [36]:
create_schema.make_data_schema_github(name_json = name_json)

# Analytics

In this part, we are providing basic summary statistic. Since we have created the tables, we can parse the schema in Glue and use our json file to automatically generates the analysis.

The cells below execute the job in the key `ANALYSIS`. You need to change the `primary_key` and `secondary_key` 

For a full analysis of the table, please use the following Lambda function. Be patient, it can takes between 5 to 30 minutes. Times varies according to the number of columns in your dataset.

Use the function as follow:

- `output_prefix`:  s3://datalake-datascience/ANALYTICS/OUTPUT/TABLE_NAME/
- `region`: region where the table is stored
- `bucket`: Name of the bucket
- `DatabaseName`: Name of the database
- `table_name`: Name of the table
- `group`: variables name to group to count the duplicates
- `primary_key`: Variable name to perform the grouping -> Only one variable for now
- `secondary_key`: Variable name to perform the secondary grouping -> Only one variable for now
- `proba`: Chi-square analysis probabilitity
- `y_var`: Continuous target variables

Check the job processing in Sagemaker: https://eu-west-3.console.aws.amazon.com/sagemaker/home?region=eu-west-3#/processing-jobs

The notebook is available: https://s3.console.aws.amazon.com/s3/buckets/datalake-datascience?region=eu-west-3&prefix=ANALYTICS/OUTPUT/&showversions=false

Please, download the notebook on your local machine, and convert it to HTML:

```
cd "/Users/thomas/Downloads/Notebook"
aws s3 cp s3://datalake-datascience/ANALYTICS/OUTPUT/asif_unzip_data_csv/Template_analysis_from_lambda-2020-11-22-08-12-20.ipynb .

## convert HTML no code
jupyter nbconvert --no-input --to html Template_analysis_from_lambda-2020-11-21-14-30-45.ipynb
jupyter nbconvert --to html Template_analysis_from_lambda-2020-11-22-08-12-20.ipynb
```

Then upload the HTML to: https://s3.console.aws.amazon.com/s3/buckets/datalake-datascience?region=eu-west-3&prefix=ANALYTICS/HTML_OUTPUT/

Add a new folder with the table name in upper case

In [ ]:
import boto3

key, secret_ = con.load_credential()
client_lambda = boto3.client(
    'lambda',
    aws_access_key_id=key,
    aws_secret_access_key=secret_,
    region_name = region)

In [ ]:
primary_key = ''
secondary_key = ''
y_var = ''

In [ ]:
payload = {
    "input_path": "s3://datalake-datascience/ANALYTICS/TEMPLATE_NOTEBOOKS/template_analysis_from_lambda.ipynb",
    "output_prefix": "s3://datalake-datascience/ANALYTICS/OUTPUT/{}/".format(table_name.upper()),
    "parameters": {
        "region": "{}".format(region),
        "bucket": "{}".format(bucket),
        "DatabaseName": "{}".format(DatabaseName),
        "table_name": "{}".format(table_name),
        "group": "{}".format(','.join(partition_keys)),
        "keys": "{},{}".format(primary_key,secondary_key),
        "y_var": "{}".format(y_var),
        "threshold":0
    },
}
payload

In [ ]:
response = client_lambda.invoke(
    FunctionName='RunNotebook',
    InvocationType='RequestResponse',
    LogType='Tail',
    Payload=json.dumps(payload),
)
response

# Generation report

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [ ]:
create_report.create_report(extension = "html", keep_code = True, notebookname =  notebookname)

In [ ]:
create_schema.create_schema(name_json, path_save_image = os.path.join(parent_path, 'utils'))

In [ ]:
### Update TOC in Github
for p in [parent_path,
          str(Path(path).parent),
          os.path.join(str(Path(path).parent), "00_download_data"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "00_statistical_exploration"),
          #os.path.join(str(Path(path).parent.parent), "02_data_analysis", "01_model_estimation"),
         ]:
    try:
        os.remove(os.path.join(p, 'README.md'))
    except:
        pass
    path_parameter = os.path.join(parent_path,'utils', name_json)
    md_lines =  make_toc.create_index(cwd = p, path_parameter = path_parameter)
    md_out_fn = os.path.join(p,'README.md')
    
    if p == parent_path:
    
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = True, path_parameter = path_parameter)
    else:
        make_toc.replace_index(md_out_fn, md_lines, Header = os.path.basename(p).replace('_', ' '), add_description = False)